In [20]:
from ibm_watsonx_ai.credentials import Credentials
from ibm_watsonx_ai.client import APIClient
from torch.distributed.pipelining import pipeline

WATSONX_APIKEY="aA0y_IlhJ9BC5__fl4rjZsyC6GoQ4T2Zrc5UFwVWwSr_"
WATSONX_URL="https://eu-de.ml.cloud.ibm.com"
WATSONX_PROJECT_ID="995ba95c-cecd-4d6f-b5cf-96dcd71d8b1e"

watsonx_client = APIClient(
        credentials=Credentials(
        url=WATSONX_URL,
        api_key=WATSONX_APIKEY
    ),
    project_id=WATSONX_PROJECT_ID)



2025-10-09 15:17:17,563 - INFO - Client successfully initialized


# 9 Punkte Plan

1. Dokument zerlegen
2. Verarbeiten
3. Kategorisieren
4. Zusammenführen
5. Metadaten
6. Abspeichern
7. Container bauen
8. Laufen lassen

----
9. Ui basteln

## 0. Standart Variablen und Settings

In [21]:
WATSONX_APIKEY="aA0y_IlhJ9BC5__fl4rjZsyC6GoQ4T2Zrc5UFwVWwSr_"
WATSONX_URL="https://eu-de.ml.cloud.ibm.com"
WATSONX_PROJECT_ID="995ba95c-cecd-4d6f-b5cf-96dcd71d8b1e"

In [22]:
pdf_file_path = "data/Dokument00.pdf"

## 1. Dokument zerlegen

In [23]:
from dokument import Single_Page

pageSpeicher: list[Single_Page] = []

In [24]:
import base64
from pymupdf import pymupdf

def pdf_to_page_images(pdf_path: str, dpi: int = 200) -> list[bytes]:
    """Rendert jede PDF-Seite als PNG-Bytes."""
    images: list[bytes] = []
    with pymupdf.open(pdf_path) as doc:
        for page in doc:
            pix = page.get_pixmap(dpi=dpi)
            img_bytes = pix.tobytes("png")
            images.append(img_bytes)
    return images



In [25]:
pdf_pages = pdf_to_page_images(pdf_file_path)
print(f"Number of pages extracted: {len(pdf_pages)}")

from dokument import Single_Page

for pageNumber, page  in enumerate(pdf_pages):
    pageSpeicher.append(Single_Page(pageNumber+1, page))

Number of pages extracted: 8


## 2. Verarbeiten

In [26]:

from docling.pipeline.vlm_pipeline import VlmPipeline
from docling.datamodel.base_models import InputFormat
from docling.document_converter import DocumentConverter, PdfFormatOption, ImageFormatOption
from docling.datamodel.pipeline_options import VlmPipelineOptions
from docling.datamodel.pipeline_options_vlm_model import ApiVlmOptions, ResponseFormat
import requests

def watsonx_vlm_options(model: str, prompt: str):
    api_key = WATSONX_APIKEY
    project_id = WATSONX_PROJECT_ID

    def _get_iam_access_token(api_key: str) -> str:
        res = requests.post(
            url="https://iam.cloud.ibm.com/identity/token",
            headers={
                "Content-Type": "application/x-www-form-urlencoded",
            },
            data=f"grant_type=urn:ibm:params:oauth:grant-type:apikey&apikey={api_key}",
        )
        res.raise_for_status()
        api_out = res.json()
        return api_out["access_token"]

    options = ApiVlmOptions(
        url=WATSONX_URL,
        params=dict(
            model_id=model,
            project_id=project_id,
            parameters=dict(
                max_new_tokens=400,
            ),
        ),
        headers={
            "Authorization": "Bearer " + _get_iam_access_token(api_key=api_key),
        },
        prompt=prompt,
        timeout=60,
        response_format=ResponseFormat.OTSL,
    )
    return options


In [27]:
from docling.datamodel import vlm_model_specs
from docling.document_converter import ImageFormatOption
import io
import tempfile
from typing import Union


def process_pdf_with_docling(page_image: Union[bytes, str, io.BytesIO]):


    # Configure the converter for image input
    converter = DocumentConverter(
        format_options={
            InputFormat.IMAGE: ImageFormatOption(),
        }
    )

    # Normalize input to a filesystem path because DocumentConverter is most
    # reliable with path-based sources across versions.
    tmp_path: str | None = None

    try:
        source_arg = None

        # If a path string was provided
        if isinstance(page_image, str) and os.path.exists(page_image):
            source_arg = page_image

        # If bytes or BytesIO was provided, persist to a temporary image file
        else:
            if isinstance(page_image, io.BytesIO):
                data = page_image.getvalue()
            elif isinstance(page_image, (bytes, bytearray)):
                data = bytes(page_image)
            else:
                raise TypeError("Unsupported type for page_image. Provide bytes, BytesIO, or file path string.")

            # Write to a temporary PNG file
            with tempfile.NamedTemporaryFile(suffix=".png", delete=False) as tmp:
                tmp.write(data)
                tmp.flush()
                tmp_path = tmp.name
                source_arg = tmp_path

        doc = converter.convert(source=source_arg).document
        return doc.export_to_doctags()
    finally:
        # Clean up the temporary file if we created one
        if tmp_path and os.path.exists(tmp_path):
            try:
                os.unlink(tmp_path)
            except OSError:
                pass

In [28]:
# einmal alles durchrechnen 
# TODO: WELCHES MODEL?

model = ""
for i, page in enumerate(pageSpeicher):
        try:
            page.docling = process_pdf_with_docling(page.Original)
            print(f"Processed page {i + 1}")
        except Exception as e:
            print(f"Error processing page {i + 1}: {str(e)}")
            

2025-10-09 15:17:18,840 - INFO - detected formats: [<InputFormat.IMAGE: 'image'>]
2025-10-09 15:17:18,870 - INFO - Going to convert document batch...
2025-10-09 15:17:18,871 - INFO - Initializing pipeline for StandardPdfPipeline with options hash e647edf348883bed75367b22fbe60347
2025-10-09 15:17:18,871 - INFO - Accelerator device: 'mps'
2025-10-09 15:17:21,884 - INFO - Accelerator device: 'mps'
2025-10-09 15:17:22,668 - INFO - Accelerator device: 'mps'
2025-10-09 15:17:23,156 - INFO - Processing document tmpha5qbrn2.png
2025-10-09 15:17:31,414 - INFO - Finished converting document tmpha5qbrn2.png in 12.57 sec.
2025-10-09 15:17:31,429 - INFO - detected formats: [<InputFormat.IMAGE: 'image'>]
2025-10-09 15:17:31,450 - INFO - Going to convert document batch...
2025-10-09 15:17:31,450 - INFO - Initializing pipeline for StandardPdfPipeline with options hash e647edf348883bed75367b22fbe60347
2025-10-09 15:17:31,450 - INFO - Accelerator device: 'mps'


Processed page 1


2025-10-09 15:17:33,824 - INFO - Accelerator device: 'mps'
2025-10-09 15:17:34,552 - INFO - Accelerator device: 'mps'
2025-10-09 15:17:35,007 - INFO - Processing document tmp5fa7yw1e.png
2025-10-09 15:17:41,864 - INFO - Finished converting document tmp5fa7yw1e.png in 10.43 sec.
2025-10-09 15:17:41,877 - INFO - detected formats: [<InputFormat.IMAGE: 'image'>]
2025-10-09 15:17:41,899 - INFO - Going to convert document batch...
2025-10-09 15:17:41,900 - INFO - Initializing pipeline for StandardPdfPipeline with options hash e647edf348883bed75367b22fbe60347
2025-10-09 15:17:41,900 - INFO - Accelerator device: 'mps'


Processed page 2


2025-10-09 15:17:44,290 - INFO - Accelerator device: 'mps'
2025-10-09 15:17:45,042 - INFO - Accelerator device: 'mps'
2025-10-09 15:17:45,476 - INFO - Processing document tmp86ssliwq.png
2025-10-09 15:17:53,221 - INFO - Finished converting document tmp86ssliwq.png in 11.34 sec.
2025-10-09 15:17:53,241 - INFO - detected formats: [<InputFormat.IMAGE: 'image'>]
2025-10-09 15:17:53,259 - INFO - Going to convert document batch...
2025-10-09 15:17:53,260 - INFO - Initializing pipeline for StandardPdfPipeline with options hash e647edf348883bed75367b22fbe60347
2025-10-09 15:17:53,260 - INFO - Accelerator device: 'mps'


...

Processed page 6


2025-10-09 15:18:18,625 - INFO - Accelerator device: 'mps'
2025-10-09 15:18:19,358 - INFO - Accelerator device: 'mps'
2025-10-09 15:18:19,627 - INFO - Processing document tmp8yao8xmd.png
2025-10-09 15:18:27,386 - INFO - Finished converting document tmp8yao8xmd.png in 11.14 sec.
2025-10-09 15:18:27,402 - INFO - detected formats: [<InputFormat.IMAGE: 'image'>]
2025-10-09 15:18:27,418 - INFO - Going to convert document batch...
2025-10-09 15:18:27,419 - INFO - Initializing pipeline for StandardPdfPipeline with options hash e647edf348883bed75367b22fbe60347
2025-10-09 15:18:27,419 - INFO - Accelerator device: 'mps'


Processed page 7


2025-10-09 15:18:29,829 - INFO - Accelerator device: 'mps'
2025-10-09 15:18:30,523 - INFO - Accelerator device: 'mps'
2025-10-09 15:18:30,914 - INFO - Processing document tmptrxc54a5.png
2025-10-09 15:18:34,342 - INFO - Finished converting document tmptrxc54a5.png in 6.94 sec.


Processed page 8


## 3. Kategorisieren

In [29]:
from ibm_watsonx_ai.foundation_models import Model, ModelInference

# Prompt-Text definieren
prompt_text = (
    "Du bist ein Assistent, der einzelne PDF-Seiten in inhaltliche Kategorien einordnet. "
    "Die Kategorien können z.b. Ausweis, Geburtsurkunde, Sterbeurkunde und Zeugnis sein"
    "Antworte ausschließlich mit einem Wort"
)

model_Kategorisierung = "meta-llama/llama-3-2-90b-vision-instruct"

model = ModelInference(
    model_id=model_Kategorisierung,
    # credentials=watsonx_client,
    credentials=Credentials(url=WATSONX_URL, api_key=WATSONX_APIKEY),
    project_id=WATSONX_PROJECT_ID
)

for i, page in enumerate(pageSpeicher):
    # Prompt erstellen und ans Model senden
    # prompt = f"{prompt_text}\n\nSeiteninhalt:\n{page.docling}\n\n\n{page.Original}"
    prompt = f"{prompt_text}\n\nSeiteninhalt:\n{page.docling}"
    response = model.generate_text(prompt)
    page.kategorisierung = response
    print(f"Kategorisierung von Seite {i + 1} abgeschlossen")

2025-10-09 15:18:34,665 - INFO - Client successfully initialized
2025-10-09 15:18:35,156 - INFO - HTTP Request: GET https://eu-de.ml.cloud.ibm.com/ml/v1/foundation_model_specs?version=2025-09-19&project_id=995ba95c-cecd-4d6f-b5cf-96dcd71d8b1e&filters=function_text_generation%2C%21lifecycle_withdrawn%3Aand&limit=200 "HTTP/1.1 200 OK"
2025-10-09 15:18:35,172 - INFO - Successfully finished Get available foundation models for url: 'https://eu-de.ml.cloud.ibm.com/ml/v1/foundation_model_specs?version=2025-09-19&project_id=995ba95c-cecd-4d6f-b5cf-96dcd71d8b1e&filters=function_text_generation%2C%21lifecycle_withdrawn%3Aand&limit=200'
2025-10-09 15:18:35,924 - INFO - HTTP Request: POST https://eu-de.ml.cloud.ibm.com/ml/v1/text/generation?version=2025-09-19 "HTTP/1.1 200 OK"
2025-10-09 15:18:35,927 - INFO - Successfully finished generate for url: 'https://eu-de.ml.cloud.ibm.com/ml/v1/text/generation?version=2025-09-19'


Kategorisierung von Seite 1 abgeschlossen


2025-10-09 15:18:36,184 - INFO - HTTP Request: POST https://eu-de.ml.cloud.ibm.com/ml/v1/text/generation?version=2025-09-19 "HTTP/1.1 200 OK"
2025-10-09 15:18:36,185 - INFO - Successfully finished generate for url: 'https://eu-de.ml.cloud.ibm.com/ml/v1/text/generation?version=2025-09-19'


Kategorisierung von Seite 2 abgeschlossen


2025-10-09 15:18:36,486 - INFO - HTTP Request: POST https://eu-de.ml.cloud.ibm.com/ml/v1/text/generation?version=2025-09-19 "HTTP/1.1 200 OK"
2025-10-09 15:18:36,487 - INFO - Successfully finished generate for url: 'https://eu-de.ml.cloud.ibm.com/ml/v1/text/generation?version=2025-09-19'


...

Kategorisierung von Seite 6 abgeschlossen


2025-10-09 15:18:38,646 - INFO - HTTP Request: POST https://eu-de.ml.cloud.ibm.com/ml/v1/text/generation?version=2025-09-19 "HTTP/1.1 200 OK"
2025-10-09 15:18:38,650 - INFO - Successfully finished generate for url: 'https://eu-de.ml.cloud.ibm.com/ml/v1/text/generation?version=2025-09-19'


Kategorisierung von Seite 7 abgeschlossen


2025-10-09 15:18:38,900 - INFO - HTTP Request: POST https://eu-de.ml.cloud.ibm.com/ml/v1/text/generation?version=2025-09-19 "HTTP/1.1 200 OK"
2025-10-09 15:18:38,901 - INFO - Successfully finished generate for url: 'https://eu-de.ml.cloud.ibm.com/ml/v1/text/generation?version=2025-09-19'


Kategorisierung von Seite 8 abgeschlossen


## 4. Zusammenführen
Wer gehört zusammen

In [30]:
def check_pages_belong_together(page_current, page_next, model_id="meta-llama/llama-3-2-90b-vision-instruct"):

    # Initialize model
    model = ModelInference(
        model_id=model_id,
        # credentials=watsonx_client,
        credentials=Credentials(url=WATSONX_URL, api_key=WATSONX_APIKEY),
        project_id=WATSONX_PROJECT_ID
    )

    # Create prompt to analyze page connection
    prompt = f"""Analyze these two consecutive pages and determine if they belong together.
    Only respond with 'yes' if they clearly belong together or 'no' if they don't.

    Page 1:
    {page_current}

    Page 2:
    {page_next}

    Do these pages belong together? Answer only with 'yes' or 'no'."""

    belongsTogether = model.generate_text(prompt)

    # Process response
    belongs_together = belongsTogether.strip().lower() == "yes"

    return belongs_together


In [31]:
for i in range(len(pageSpeicher) - 1):
    current_page = pageSpeicher[i]
    next_page = pageSpeicher[i + 1]

    if check_pages_belong_together(current_page.docling, next_page.docling):
        current_page.related_pages.append(next_page.page_number)


2025-10-09 15:18:39,255 - INFO - Client successfully initialized
2025-10-09 15:18:39,859 - INFO - HTTP Request: GET https://eu-de.ml.cloud.ibm.com/ml/v1/foundation_model_specs?version=2025-09-19&project_id=995ba95c-cecd-4d6f-b5cf-96dcd71d8b1e&filters=function_text_generation%2C%21lifecycle_withdrawn%3Aand&limit=200 "HTTP/1.1 200 OK"
2025-10-09 15:18:39,870 - INFO - Successfully finished Get available foundation models for url: 'https://eu-de.ml.cloud.ibm.com/ml/v1/foundation_model_specs?version=2025-09-19&project_id=995ba95c-cecd-4d6f-b5cf-96dcd71d8b1e&filters=function_text_generation%2C%21lifecycle_withdrawn%3Aand&limit=200'
2025-10-09 15:18:40,324 - INFO - HTTP Request: POST https://eu-de.ml.cloud.ibm.com/ml/v1/text/generation?version=2025-09-19 "HTTP/1.1 200 OK"
2025-10-09 15:18:40,326 - INFO - Successfully finished generate for url: 'https://eu-de.ml.cloud.ibm.com/ml/v1/text/generation?version=2025-09-19'
2025-10-09 15:18:40,549 - INFO - Client successfully initialized
2025-10-09 1

In [32]:
# Initialize list to store documents
documents = []

# Process each page in pageSpeicher
i = 0
while i < len(pageSpeicher):
    current_doc = [pageSpeicher[i]]
    j = i + 1

    # Check subsequent pages if they belong to current document
    while j < len(pageSpeicher):
        if pageSpeicher[j].page_number in pageSpeicher[j - 1].related_pages:
            current_doc.append(pageSpeicher[j])
            j += 1
        else:
            break

    # Add current document to documents list
    documents.append(current_doc)
    i = j

# Print number of documents found
print(f"Found {len(documents)} documents")


Found 8 documents


In [33]:
from PyPDF2 import PdfWriter, PdfReader
import io
from PIL import Image
import os


def save_documents_as_pdfs(documents, output_dir="output_pdfs"):
    """
    Save each document (group of pages) as a separate PDF file.
    
    Args:
        documents: List of lists containing Single_Page objects
        output_dir: Directory where PDF files will be saved
    """

    # Create output directory if it doesn't exist
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Process each document
    for doc_idx, document in enumerate(documents):
        pdf_writer = PdfWriter()

        # Process each page in the document
        for page in document:
            # Convert PNG bytes to PIL Image
            img = Image.open(io.BytesIO(page.Original))

            # Convert to PDF
            pdf_bytes = io.BytesIO()
            img.save(pdf_bytes, format='PDF')
            pdf_bytes.seek(0)

            # Create PDF reader from bytes and add page to writer
            pdf_reader = PdfReader(pdf_bytes)
            pdf_writer.add_page(pdf_reader.pages[0])

        # Save the combined PDF
        output_path = os.path.join(output_dir, f"document_{doc_idx + 1}.pdf")
        with open(output_path, "wb") as output_file:
            pdf_writer.write(output_file)

    print(f"Saved {len(documents)} PDF documents to {output_dir}/")


In [34]:
save_documents_as_pdfs(documents, "data/output")

Saved 8 PDF documents to data/output/


## 5.Metadaten rausholen

## 6. Abspeichern

In [35]:
from cloudant.client import Cloudant
from cloudant.error import CloudantException
import json


def save_to_cloudant(documents, db_name="documents_db"):
    # Initialize the Cloudant client
    # Replace with your Cloudant credentials
    client = Cloudant.iam(
        account_name="2299df37-27b3-44b3-bf1a-f7ca1996fc92-bluemix",
        api_key="CMN7PgnNApziV6Yq1KcmJD1671DbuDleUY7IhHDYIska",
        connect=True
    )

    # Create database if it doesn't exist
    if db_name not in client.all_dbs():
        client.create_database(db_name)

    db = client[db_name]

    # Process each document
    for doc_idx, document in enumerate(documents):
        # Create document data
        doc_data = {
            "type": "document",
            "doc_id": f"doc_{doc_idx + 1}",
            "pages": []
        }

        # Add data for each page
        for page in document:
            page_data = {
                "page_number": page.page_number,
                "docling_content": page.docling,
                "categorization": page.kategorisierung,
                "related_pages": page.related_pages
                # "original_content": page.Original
            }
            doc_data["pages"].append(page_data)

        # Save document to database
        try:
            doc = db.create_document(doc_data)
            print(f"Saved document {doc_idx + 1} with id: {doc['_id']}")
        except CloudantException as e:
            print(f"Error saving document {doc_idx + 1}: {str(e)}")

    client.disconnect()


In [36]:
save_to_cloudant(documents)

Saved document 1 with id: d73af8a5c0c74a9d44fe24460ca73acf
Saved document 2 with id: 71914736f57c3047d35523b5118e0ea8
Saved document 3 with id: ca8ae5d8732cd52bb98eb7269878f783
Saved document 4 with id: 0ad5bb260e8a76e8adc2888356adb1e4
Saved document 5 with id: 77f9025844327eca6c8256cb4f9dac8a
Saved document 6 with id: 150e741867ba9e36518118459c754bc8
Saved document 7 with id: b4ba57f93320a85b1c141b4a60e31324
Saved document 8 with id: 0ad5bb260e8a76e8adc2888356adc185
